# Colab-VSGAN

My repo: [styler00dollar/VSGAN-tensorrt-docker](https://github.com/styler00dollar/VSGAN-tensorrt-docker/)

In [ ]:
!nvidia-smi

In [ ]:
!nvcc --version

In [ ]:
# check python version, for me its currently 3.7.12
!python --version

In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
#@title Install TensorRT 8.4.x
#@markdown Warning: 8.4 is currently not compatible with `Torch-TensorRT`! Use `vs-mlrt` instead, which is faster anyway and you don't need an account like the other install process.
%cd /content
!sudo apt-get update
!sudo apt-get install libnvinfer8 libnvonnxparsers8 libnvparsers8 libnvinfer-plugin8 libnvinfer-dev libnvonnxparsers-dev libnvparsers-dev libnvinfer-plugin-dev python3-libnvinfer tensorrt python3-libnvinfer-dev

In [ ]:
#@title (Not recommended) Install TensorRT 8.2.x
#@markdown If you really really want `Torch-TensorRT` instead of `vs-mlrt`, then do the following:

#@markdown You need to get 2 files.
#@markdown `nv-tensorrt-repo-ubuntu1804-cuda11.4-trt8.2.3.0-ga-20220113_1-1_amd64.deb` from
#@markdown `(TensorRT 8.2 GA Update 2 for Ubuntu 18.04 and CUDA 11.0, 11.1, 11.2, 11.3, 11.4 and 11.5 DEB local repo Package)`
#@markdown and `TensorRT-8.2.3.0.Linux.x86_64-gnu.cuda-11.4.cudnn8.2.tar.gz/python/tensorrt-8.2.3.0-cp37-none-linux_x86_64.whl`
#@markdown from `(TensorRT 8.2 GA Update 2 for Linux x86_64 and CUDA 11.0, 11.1, 11.2, 11.3, 11.4 and 11.5 TAR Package)`
#@markdown  You can download these files [here](https://developer.nvidia.com/nvidia-tensorrt-download). Warning: You need an account (which can be created for free). 
#@markdown Put them into `gdrive/TensorRT`.

import os
os.environ["os1"]="ubuntu1804"
os.environ["tag"]= "cuda11.4-trt8.2.3.0-ga-20220113" #@param
os.environ["version"]= "8.2.3-1+cuda11.4" #@param
data_path = '/content/drive/MyDrive/TensorRT/' #@param
os.chdir(data_path)
!sudo dpkg -i nv-tensorrt-repo-${os1}-${tag}_1-1_amd64.deb
!sudo apt-key add /var/nv-tensorrt-repo-${tag}/7fa2af80.pub
!sudo apt-get update
!sudo apt-get install libnvinfer8=${version} libnvonnxparsers8=${version} libnvparsers8=${version} libnvinfer-plugin8=${version} libnvinfer-dev=${version} libnvonnxparsers-dev=${version} libnvparsers-dev=${version} libnvinfer-plugin-dev=${version} python3-libnvinfer=${version}
!sudo apt-mark hold libnvinfer8 libnvonnxparsers8 libnvparsers8 libnvinfer-plugin8 libnvinfer-dev libnvonnxparsers-dev libnvparsers-dev libnvinfer-plugin-dev python3-libnvinfer
!sudo apt-get install tensorrt=8.2.3.0-1+cuda11.4
!sudo apt-get install python3-libnvinfer-dev=${version}

# install the whl with the matching python version
whl_path = "/content/drive/MyDrive/TensorRT/tensorrt-8.2.3.0-cp37-none-linux_x86_64.whl" #@param
!pip install {whl_path}

In [ ]:
#@title initial setup
%cd /content/
!sudo rm -rf /workspace
!mkdir -p /workspace/tensorrt
%cd /workspace/tensorrt
!git clone https://github.com/styler00dollar/VSGAN-tensorrt-docker

!pip install kornia opencv-python pytorch-msssim

In [ ]:
#@title compile vs
# installing vapoursynth related stuff
!apt install ffmpeg autoconf libtool yasm python3.9 python3.9-venv python3.9-dev ffmsindex libffms2-4 libffms2-dev -y
!apt --fix-broken install
# zimg
!wget https://github.com/sekrit-twc/zimg/archive/refs/tags/release-3.0.4.zip && 7z x release-3.0.4.zip
%cd zimg-release-3.0.4
!./autogen.sh && ./configure && make -j4 && make install
# vapoursynth
!pip install Cython -U --force-reinstall
!wget https://github.com/vapoursynth/vapoursynth/archive/refs/tags/R59.zip -P /workspace/tensorrt
%cd /workspace/tensorrt/
!7z x /workspace/tensorrt/R59.zip
%cd /workspace/tensorrt/vapoursynth-R59
!./autogen.sh && ./configure && make && make install && cd .. && ldconfig
!ln -s /usr/local/lib/python3.7/site-packages/vapoursynth.so /usr/lib/python3.7/lib-dynload/vapoursynth.so
!pip install vapoursynth

In [ ]:
#@title download vs
%cd /content
!apt install ffmpeg autoconf libtool yasm python3.9 python3.9-venv python3.9-dev ffmsindex libffms2-4 libffms2-dev -y
!apt --fix-broken install
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/vs_stuff.7z
!7z x vs_stuff.7z
%cd /content/vs_stuff
!sudo cp /content/vs_stuff/vspipe /usr/local/bin/vspipe
!sudo cp /content/vs_stuff/libzimg.so /usr/local/lib/libzimg.so
!sudo cp /content/vs_stuff/libvapoursynth.so /usr/local/lib/libvapoursynth.so
!sudo cp /content/vs_stuff/libvapoursynth-script.so /usr/local/lib/libvapoursynth-script.so
!pip install vapoursynth

Optional stuff, which you may or may not need:

In [ ]:
#@title install cupy / pycuda
!curl https://colab.chainer.org/install | sh -
!pip install pycuda

In [ ]:
#@title install torch trt (TensorRT8.2 only)
# pytorch tensorrt
!pip install https://github.com/pytorch/TensorRT/releases/download/v1.1.0/torch_tensorrt-1.1.0-cp37-cp37m-linux_x86_64.whl

In [ ]:
#@title install onnx
!pip install onnx onnxruntime onnxruntime-gpu

In [ ]:
#@title install onnx trt
# installing onnx tensorrt with a workaround, error with import otherwise
%cd /workspace/tensorrt
!git clone https://github.com/onnx/onnx-tensorrt
%cd onnx-tensorrt
!python3 setup.py install

In [ ]:
#@title install swinir
# vs plugings from others
# https://github.com/HolyWu/vs-swinir
!pip install --upgrade vsswinir && python -m vsswinir

In [ ]:
#@title compile mmcv + install vsbasicvsrpp (basicvsrpp/RealBasicVSR)
# https://github.com/HolyWu/vs-basicvsrpp
!pip install --upgrade vsbasicvsrpp && python -m vsbasicvsrpp
# dependencies for RealBasicVSR_x4
# mmedit
!git clone https://github.com/open-mmlab/mmediting.git && cd mmediting && pip install -v -e .
# RealBasicVSR_x4 will download this
!wget "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" -P /root/.cache/torch/hub/checkpoints/

In [ ]:
#@title download models
%cd /workspace
# downloading models
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/RealESRGANv2-animevideo-xsx2.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/RealESRGANv2-animevideo-xsx4.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/RealESRGAN_x4plus_anime_6B.pth
# fatal anime
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/4x_fatal_Anime_500000_G.pth
# rvp1
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/rvpV1_105661_G.pt
# sepconv
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/sepconv.pth
# EGVSR
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/EGVSR_iter420000.pth
# rife4 (fixed rife4.0 model)
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/rife40.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/rife41.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/rife42.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/rife43.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/sudo_rife4_269.662_testV1_scale1.pth
# RealBasicVSR_x4
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/RealBasicVSR_x4.pth
# cugan models
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/cugan_up2x-latest-conservative.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/cugan_up2x-latest-denoise1x.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/cugan_up2x-latest-denoise2x.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/cugan_up2x-latest-denoise3x.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/cugan_up2x-latest-no-denoise.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/cugan_up3x-latest-conservative.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/cugan_up3x-latest-denoise3x.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/cugan_up3x-latest-no-denoise.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/cugan_up4x-latest-conservative.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/cugan_up4x-latest-denoise3x.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/cugan_up4x-latest-no-denoise.pth

!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/cugan_pro-conservative-up2x.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/cugan_pro-conservative-up3x.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/cugan_pro-denoise3x-up2x.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/cugan_pro-denoise3x-up3x.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/cugan_pro-no-denoise3x-up2x.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/cugan_pro-no-denoise3x-up3x.pth
# IFRNet
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/IFRNet_S_Vimeo90K.pth
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/IFRNet_L_Vimeo90K.pth
# film
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/FILM.tar.gz
!tar -zxvf FILM.tar.gz

Vulkan related:

In [ ]:
#@title install vulkan
!sudo apt-get install -y libvulkan-dev nasm
!pip install meson ninja
# upgrading g++
!sudo apt install build-essential manpages-dev software-properties-common -y
!sudo add-apt-repository ppa:ubuntu-toolchain-r/test -y
!sudo apt update -y && sudo apt install gcc-11 g++-11 -y
!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-11 11
!sudo update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-11 11

In [ ]:
#@title compile rife ncnn (needs compiled vs)
%cd /content/
!rm -rf vmaf-2.3.1
!wget https://github.com/Netflix/vmaf/archive/refs/tags/v2.3.1.tar.gz
# VMAF
!tar -xzf  v2.3.1.tar.gz
%cd vmaf-2.3.1/libvmaf/
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 meson build --buildtype release
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 ninja -C build && ninja -C build install

%cd /content
!rm -rf VapourSynth-VMAF
!git clone https://github.com/HomeOfVapourSynthEvolution/VapourSynth-VMAF
%cd VapourSynth-VMAF
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 meson build
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 ninja -C build && ninja -C build install

# MISC
%cd /content
!rm -rf vs-miscfilters-obsolete
!git clone https://github.com/vapoursynth/vs-miscfilters-obsolete
%cd vs-miscfilters-obsolete
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 meson build
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 ninja -C build && ninja -C build install
    
# RIFE
%cd /content
!rm -rf VapourSynth-RIFE-ncnn-Vulkan
!git clone https://github.com/styler00dollar/VapourSynth-RIFE-ncnn-Vulkan
%cd VapourSynth-RIFE-ncnn-Vulkan
!git submodule update --init --recursive --depth 1
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 meson build -Dbuildtype=debug -Db_lto=false
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 ninja -C build && ninja -C build install

In [ ]:
#@title download compiled rife ncnn
%cd /content

!git clone https://github.com/styler00dollar/VapourSynth-RIFE-ncnn-Vulkan
!mkdir -p /usr/local/lib/vapoursynth/models/
!sudo cp -r /content/VapourSynth-RIFE-ncnn-Vulkan/models /usr/local/lib/vapoursynth/

%cd /content
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/compiled_rife_ncnn.7z
!7z x compiled_rife_ncnn.7z

!mkdir -p /usr/local/lib/x86_64-linux-gnu/
!cp /content/compiled_rife_ncnn/libvmaf.so /usr/local/lib/vapoursynth/libvmaf.so
!cp /content/compiled_rife_ncnn/libmiscfilters.so /usr/local/lib/vapoursynth/libmiscfilters.so 
!cp /content/compiled_rife_ncnn/librife.so /usr/local/lib/vapoursynth/librife.so 
!cp /content/compiled_rife_ncnn/vmaf-2.3.1/libvmaf.so /usr/local/lib/x86_64-linux-gnu/libvmaf.so
!cp /content/compiled_rife_ncnn/vmaf-2.3.1/libvmaf.so.1 /usr/local/lib/x86_64-linux-gnu/libvmaf.so.1
!cp /content/compiled_rife_ncnn/vmaf-2.3.1/libvmaf.so.1.1.3 /usr/local/lib/x86_64-linux-gnu/libvmaf.so.1.1.3
!cp /content/compiled_rife_ncnn/vmaf-2.3.1/libvmaf.a /usr/local/lib/x86_64-linux-gnu/libvmaf.a
!sudo ldconfig

In [ ]:
#@title compile vs-mlrt
# updating cmake
%cd /content
!wget https://github.com/Kitware/CMake/releases/download/v3.24.0-rc1/cmake-3.24.0-rc1-linux-x86_64.sh
!chmod +x cmake-3.24.0-rc1-linux-x86_64.sh
!sh cmake-3.24.0-rc1-linux-x86_64.sh --skip-license
!cp /content/bin/cmake /usr/bin/cmake
!cp /content/bin/cmake /usr/lib/x86_64-linux-gnu/cmake
!cp /content/bin/cmake /usr/local/bin/cmake
!cp -r /content/share/cmake-3.24 /usr/local/share/
# upgrading g++
!sudo apt install build-essential manpages-dev software-properties-common -y
!sudo add-apt-repository ppa:ubuntu-toolchain-r/test -y
!sudo apt update -y && sudo apt install gcc-11 g++-11 -y
!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-11 11
!sudo update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-11 11
# getting vapoursynth sourcecode
%cd /content
!wget https://github.com/vapoursynth/vapoursynth/archive/refs/tags/R59.zip
!7z x R59.zip
# compiling
%cd /content/
!sudo rm -rf vs-mlrt
!git clone https://github.com/AmusementClub/vs-mlrt
%cd vs-mlrt/vstrt
!mkdir build
%cd build
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 cmake .. -DVAPOURSYNTH_INCLUDE_DIRECTORY=/content/vapoursynth-R59/include
!make
!sudo make install
%cd /content
#!wget https://github.com/AmusementClub/vs-mlrt/releases/download/v7/models.v7.7z
#!7z x models.v7.7z

Rendering related:

In [ ]:
#@title install x264
# render tools
!sudo apt install x264 -y

In [ ]:
#@title compile x265
!sudo apt-get install -y numactl
# updating cmake
%cd /content
!wget https://github.com/Kitware/CMake/releases/download/v3.23.0-rc1/cmake-3.23.0-rc1-linux-x86_64.sh
!chmod +x cmake-3.23.0-rc1-linux-x86_64.sh
!sh cmake-3.23.0-rc1-linux-x86_64.sh --skip-license
!cp /content/bin/cmake /usr/bin/cmake
!cp /content/bin/cmake /usr/lib/x86_64-linux-gnu/cmake
!cp /content/bin/cmake /usr/local/bin/cmake
!cp -r /content/share/cmake-3.23 /usr/local/share/
# upgrading g++
!sudo apt install build-essential manpages-dev software-properties-common -y
!sudo add-apt-repository ppa:ubuntu-toolchain-r/test -y
!sudo apt update -y && sudo apt install gcc-11 g++-11 -y
!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-11 11
!sudo update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-11 11
# compile x265
%cd /content
!sudo rm -rf x265
!git clone https://github.com/AmusementClub/x265
%cd x265/source/
!mkdir build
%cd build
!cmake .. -DNATIVE_BUILD=ON -DSTATIC_LINK_CRT=ON -DENABLE_AVISYNTH=OFF
!make 
!sudo make install
!cp /content/x265/source/build/x265 /usr/bin/x265 
!cp /content/x265/source/build/x265 /usr/local/bin/x265

`Runtime -> Restart runtime`

# Render

In [ ]:
# if you want to use vs-mlrt, convert model into engine, you can find models in /content/models
# after creating the model, you can set the path in inference.py
%cd /content
!/usr/src/tensorrt/bin/trtexec --fp16 --onnx=/content/model.onnx --minShapes=input:1x3x8x8 --optShapes=input:1x3x720x1280 --maxShapes=input:1x3x1080x1920 --saveEngine=/content/model.engine --tacticSources=+CUDNN,+CUBLAS,+CUBLAS_LT

In [ ]:
#@title inference.py
%%writefile /workspace/tensorrt/VSGAN-tensorrt-docker/inference.py
import sys

sys.path.append('/workspace/tensorrt/VSGAN-tensorrt-docker/')
import vapoursynth as vs

# select what you installed/want
"""
from src.esrgan import ESRGAN_inference  # esrgan and realesrgan
from src.SRVGGNetCompact import SRVGGNetCompactRealESRGAN  # realesrgan anime video
from src.vfi_model import (
    video_model,
)  # any vfi model, in this case rvp1 as demonstration
from src.sepconv_enhanced import sepconv_model  # uses cupy, no tensorrt
from src.rife import RIFE  # tensorrt not possible
from vsswinir import (
    SwinIR,
)  # https://github.com/HolyWu/vs-swinir # currently not tensorrt, didn't try
from src.egvsr import egvsr_model  # currently not tensorrt
from src.cugan import cugan_inference
from vsbasicvsrpp import BasicVSRPP
from src.realbasicvsr import realbasicvsr_model
from src.film import FILM_inference
from src.pan import PAN_inference
from src.IFRNet import IFRNet
from src.M2M import M2M
from src.IFUNet import IFUNet
from src.eisai import EISAI
"""
from src.rife import RIFE  # tensorrt not possible

core = vs.core
vs_api_below4 = vs.__api_version__.api_major < 4
core = vs.core
core.num_threads = 4  # can influence ram usage

# select what you installed/want
core.std.LoadPlugin(path='/usr/lib/x86_64-linux-gnu/libffms2.so')
# load if compiled
#core.std.LoadPlugin(path='/usr/local/lib/libvstrt.so')

video_path = "480.mkv"
# cfr video
clip = core.ffms2.Source(source=video_path)
# vfr video (untested)
# clip = core.ffms2.Source(source='input.mkv', fpsnum = 24000, fpsden = 1001)

# resizing with descale
# Debilinear, Debicubic, Delanczos, Despline16, Despline36, Despline64, Descale
# clip = core.descale.Debilinear(clip, 1280, 720)

###############################################
# COLORSPACE
###############################################
# convert colorspace
clip = vs.core.resize.Bicubic(clip, format=vs.RGBS, matrix_in_s="709")
# convert colorspace + resizing
#clip = vs.core.resize.Bicubic(
#    clip, width=848, height=480, format=vs.RGBS, matrix_in_s="709"
#)

###############################################

# these demos work out of the box because docker also downloads the needed models, if you want other models, just add them
# you can combine everything however you want

###############################################
# MODELS (CUDA)
###############################################
# sepconv
# clip = sepconv_model(clip)
# RIFE4
# rife4 can do cuda and ncnn, but only cuda is supported in docker
# models: rife40 | rife41 | sudo_rife4
# only use sudo_rife4 for 2x
#clip = RIFE(
#    clip,
#    multi=2,
#    scale=1.0,
#    fp16=False,
#    fastmode=False,
#    ensemble=True,
#    model_version="rife40",
#    psnr_dedup=False,
#    psnr_value=70,
#    ssim_dedup=False,
#    ms_ssim_dedup=False,
#    ssim_value=0.999,
#    backend_inference="cuda",
#)
# VFI example for jit models
# clip = video_model(clip, fp16=False, model_path="/workspace/rvpV1_105661_G.pt")
# SwinIR
# clip = SwinIR(clip, task="lightweight_sr", scale=2)
# ESRGAN / RealESRGAN
# tta_mode 1-7, means the amount of times the image gets processed while being mirrored
#clip = ESRGAN_inference(
#    clip=clip,
#    model_path="/workspace/4x_fatal_Anime_500000_G.pth",
#    tile_x=400,
#    tile_y=400,
#    tile_pad=10,
#    fp16=False,
#    tta=False,
#    tta_mode=1,
#)
# clip = ESRGAN_inference(
#    clip=clip,
#    model_path="/workspace/RealESRGAN_x4plus_anime_6B.pth",
#    tile_x=480,
#    tile_y=480,
#    tile_pad=16,
#    fp16=False,
#    tta=False,
#    tta_mode=1,
# )
# RealESRGAN Anime Video example
# backends: tensorrt, cuda, onnx, quantized_onnx
# clip = SRVGGNetCompactRealESRGAN(clip, scale=2, fp16=True, backend_inference="tensorrt")
# EGVSR
# clip = egvsr_model(clip, interval=15)
# BasicVSR++
# 0 = REDS, 1 = Vimeo-90K (BI), 2 = Vimeo-90K (BD), 3 = NTIRE 2021 - Track 1, 4 = NTIRE 2021 - Track 2, 5 = NTIRE 2021 - Track 3
# clip = BasicVSRPP(
#    clip,
#    model=1,
#    interval=30,
#    tile_x=0,
#    tile_y=0,
#    tile_pad=16,
#    device_type="cuda",
#    device_index=0,
#    fp16=False,
#    cpu_cache=False,
# )
# RealBasicVSR
# clip = realbasicvsr_model(clip, interval=15, fp16=True)
# cugan
# scales: 2 | 3 | 4, kind_model: no_denoise | denoise3x | conservative, backend_inference: cuda | onnx, pro: True/False (only available for 2x and 3x scale)
# only cuda supports tiling
# clip = cugan_inference(
#    clip,
#    fp16=True,
#    scale=2,
#    kind_model="no_denoise",
#    backend_inference="cuda",
#    tile_x=512,
#    tile_y=512,
#    tile_pad=10,
#    pre_pad=0,
# )
# FILM
# models: l1 | vgg | style
# clip = FILM_inference(clip, model_choise="vgg")
# vs-mlrt (you need to create the engine yourself)
# clip = core.trt.Model(
#    clip,
#    engine_path="/content/model.engine",
#    tilesize=[854, 480],
#    num_streams=3,
#)
# vs-mlrt (DPIR)
# DPIR does need an extra channel, replace BlankClip with grayscale clip
# sigma = 10.0
# noise_level_map = core.std.BlankClip(clip, width=1280, height=720, format=vs.GRAYS)
# clip = core.trt.Model(
#    [clip, core.std.BlankClip(noise_level_map, color=sigma / 255.0)],
#    engine_path="model.engine",
#    tilesize=[1280, 720],
#    num_streams=2,
# )
# PAN
# scale = 2 | 3 | 4
# clip = PAN_inference(clip, scale=2, fp16=True)
# IFRNet
# model: small | large
# clip = IFRNet(clip, model="small")
# M2M
# clip = M2M(clip, multi=4)
# IFUnet
# clip = IFUNet(clip)
# EISAI (needs 960x540 for now)
# clip = EISAI(clip)

###############################################
# NCNN
###############################################
# from src.SRVGGNetCompact_ncnn import SRVGGNetCompactRealESRGAN_ncnn

# Rife ncnn (C++)
# https://github.com/styler00dollar/VapourSynth-RIFE-ncnn-Vulkan
clip = core.misc.SCDetect(clip=clip, threshold=0.100)
clip = core.rife.RIFE(
    clip,
    model=9,
    multiplier=2,
    gpu_id=0,
    gpu_thread=4,
    tta=False,
    uhd=False,
    skip=False,
    sc=True,
)

# Rife ncnn (python api)
# clip = RIFE(
#    clip,
#    multi=2,
#    scale=1.0,
#    fp16=True,
#    fastmode=False,
#    ensemble=True,
#    psnr_dedup=False,
#    psnr_value=70,
#    ssim_dedup=True,
#    ms_ssim_dedup=False,
#    ssim_value=0.999,
#    backend_inference="ncnn",
# )

###############################################
# Deduplicated inference for faster inference
# only use this for upscaling
###############################################
# from src.dedup import return_frames

# frames_duplicated, frames_duplicating = return_frames(video_path, psnr_value=50)
# clip = core.std.DeleteFrames(clip, frames_duplicated)
# do upscaling here
# clip = core.std.DuplicateFrames(clip, frames_duplicating)
###############################################
# Inference with scene detection
# only use this for frame interpolation
###############################################
# from src.scene_detect import find_scenes
# skip_framelist = find_scenes(video_path, threshold=30)
# clip = RIFE(
#    clip,
#    multi=2,
#    scale=1.0,
#    fp16=True,
#    fastmode=False,
#    ensemble=True,
#    psnr_dedup=False,
#    psnr_value=70,
#    ssim_dedup=True,
#    ms_ssim_dedup=False,
#    ssim_value=0.999,
#    skip_framelist=skip_framelist,
# )
###############################################
# OUTPUT
###############################################
clip = vs.core.resize.Bicubic(clip, format=vs.YUV420P8, matrix_s="709")
clip.set_output()


In [ ]:
%cd /workspace/tensorrt/VSGAN-tensorrt-docker/
!sudo rm -rf /workspace/tensorrt/VSGAN-tensorrt-docker/example.mkv
!vspipe -c y4m inference.py - | ffmpeg -i pipe: /workspace/tensorrt/VSGAN-tensorrt-docker/example.mkv

In [ ]:
%cd /workspace/tensorrt/VSGAN-tensorrt-docker/
!sudo rm -rf /workspace/tensorrt/VSGAN-tensorrt-docker/example.mkv
!vspipe -c y4m inference.py - | x264 - --demuxer y4m -o /workspace/tensorrt/VSGAN-tensorrt-docker/example.mkv

In [ ]:
%cd /workspace/tensorrt/VSGAN-tensorrt-docker/
!sudo rm -rf /workspace/tensorrt/VSGAN-tensorrt-docker/example.mkv
!vspipe -c y4m inference.py - | x265 - --y4m --crf 23 -o /workspace/tensorrt/VSGAN-tensorrt-docker/example.mkv